## Stochastic TradingView Indicator

>Developed by [@edyatl](https://github.com/edyatl) April 2023 <edyatl@yandex.ru>

In [1]:
# Load Jupyter extension for auto correction coding style based on Black Lib
%load_ext nb_black

<IPython.core.display.Javascript object>

### Original Indicator code

```python
//@version=5 
indicator(title="Stochastic", shorttitle="Stoch", format=format.price, precision=2, timeframe="", timeframe_gaps=true) 

periodK = input.int(14, title="%K Length", minval=1) 
smoothK = input.int(1, title="%K Smoothing", minval=1) 
periodD = input.int(3, title="%D Smoothing", minval=1) 

k = ta.sma(ta.stoch(close, high, low, periodK), smoothK) 
d = ta.sma(k, periodD) 

plot(k, title="%K", color=#2962FF) 
plot(d, title="%D", color=#FF6D00) 

h0 = hline(80, "Upper Band", color=#787B86) 

hline(50, "Middle Band", color=color.new(#787B86, 50)) 

h1 = hline(20, "Lower Band", color=#787B86) 

fill(h0, h1, color=color.rgb(33, 150, 243, 90), title="Background")
```

In [2]:
# Standard imports
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
import talib as tl

import os
from os import environ as env
from dotenv import load_dotenv
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager

<IPython.core.display.Javascript object>

In [3]:
# Install a pip package in the current Jupyter kernel
# import sys

# !{sys.executable} -m pip install -U python-dotenv

<IPython.core.display.Javascript object>

In [4]:
project_dotenv = os.path.join(os.path.abspath(""), ".env")
if os.path.exists(project_dotenv):
    load_dotenv(project_dotenv)

<IPython.core.display.Javascript object>

In [5]:
api_key, api_secret = env.get("ENV_API_KEY"), env.get("ENV_SECRET_KEY")
client = Client(api_key, api_secret)

<IPython.core.display.Javascript object>

In [6]:
klines = client.get_klines(symbol="ATOMUSDT", interval=Client.KLINE_INTERVAL_15MINUTE)

short_col_names = [
    "open_time",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "close_time",
    "qav",
    "num_trades",
    "taker_base_vol",
    "taker_quote_vol",
    "ignore",
]

<IPython.core.display.Javascript object>

In [7]:
data = pd.DataFrame(klines, columns=short_col_names)
data["open_time"] = pd.to_datetime(data["open_time"], unit="ms")
data["close_time"] = pd.to_datetime(data["close_time"], unit="ms")
data.tail(5)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
495,2023-04-18 20:00:00,12.44900000,12.46900000,12.41300000,12.46900000,13386.76000000,2023-04-18 20:14:59.999,166542.53569000,631,7758.90000000,96509.48904000,0
496,2023-04-18 20:15:00,12.46900000,12.51300000,12.45400000,12.50900000,21220.27000000,2023-04-18 20:29:59.999,264925.52123000,829,12346.53000000,154158.51502000,0
497,2023-04-18 20:30:00,12.50900000,12.52300000,12.48400000,12.52300000,12025.86000000,2023-04-18 20:44:59.999,150291.59372000,544,6521.10000000,81499.38387000,0
498,2023-04-18 20:45:00,12.52400000,12.55600000,12.51800000,12.54600000,12002.70000000,2023-04-18 20:59:59.999,150451.06450000,576,6851.67000000,85869.60959000,0
499,2023-04-18 21:00:00,12.54500000,12.55000000,12.53300000,12.54100000,3069.93000000,2023-04-18 21:14:59.999,38497.02004000,128,1942.36000000,24356.23417000,0


<IPython.core.display.Javascript object>

### Inputs

In [8]:
periodK: int = 14  # input.int(14, title="%K Length", minval=1)
smoothK: int = 2  # input.int(3, title="%K Smoothing", minval=1)
periodD: int = 3  # input.int(3, title="%D Smoothing", minval=1)

<IPython.core.display.Javascript object>

In [9]:
k, d = tl.STOCH(
    data["high"],
    data["low"],
    data["close"],
    periodK,
    smoothK,
    0,
    periodD,
    0,
)

<IPython.core.display.Javascript object>

In [11]:
res = pd.DataFrame(
    {
        "open_time": data["open_time"],
        "%K": k,
        "%D": d,
        "high": data["high"],
        "low": data["low"],
        "close": data["close"],
    }
)
res.tail(20)

,open_time,%K,%D,high,low,close
480,2023-04-18 16:15:00,36.923077,29.369547,12.60900000,12.58200000,12.58600000
481,2023-04-18 16:30:00,26.465768,31.756658,12.59000000,12.54500000,12.56700000
482,2023-04-18 16:45:00,22.164948,28.517931,12.59000000,12.55000000,12.56900000
483,2023-04-18 17:00:00,22.680412,23.770376,12.59300000,12.55900000,12.56900000
484,2023-04-18 17:15:00,13.708627,19.517996,12.57800000,12.51000000,12.51900000
485,2023-04-18 17:30:00,4.783880,13.724307,12.52900000,12.44000000,12.45000000
486,2023-04-18 17:45:00,4.831816,7.774774,12.45600000,12.37200000,12.38500000
487,2023-04-18 18:00:00,11.066876,6.894191,12.41300000,12.35200000,12.40200000
488,2023-04-18 18:15:00,34.775087,16.891259,12.50400000,12.40300000,12.50300000
489,2023-04-18 18:30:00,47.914451,31.252138,12.51400000,12.45000000,12.46400000


<IPython.core.display.Javascript object>